In [1]:
#importing necessary libraries
from __future__ import print_function
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
%matplotlib inline  
from imageai.Detection import VideoObjectDetection
import os
import sys
from random import randint
from math import ceil, sqrt
import natsort
import pandas as pd
import random
from sklearn.svm import SVC 
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import GridSearchCV 
import pickle

Using TensorFlow backend.


In [2]:
user = 'siddhi'

if user == 'siddhi':
    path_videos = 'C:/RoadCrossingAssistant/Data/Videos/'
    path_labels_csv = 'C:/RoadCrossingAssistant/Data/labels_framewise_csv.csv'
    path_labels_list = 'C:/RoadCrossingAssistant/Data/labels_framewise_list.pkl'

elif user == 'yagnesh':
    path_videos = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos/'
    path_labels_csv = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/labels_framewise.csv'
    path_labels_list = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/labels_framewise.pkl'

videos = glob.glob(path_videos+'video*.MOV')
videos = natsort.natsorted(videos)

# frame-wise labels array
open_file = open(path_labels_list, "rb")
labels_list = pickle.load(open_file)
open_file.close()

In [3]:
#Perform train-test split(80-24)

x = np.arange(104)
#np.random.seed(42)
indices_test = np.random.choice(x, 24, replace=False)
indices_train = np.delete(x, indices_test, axis=0)

videos_train = [videos[ind] for ind in indices_train]
videos_test = [videos[ind] for ind in indices_test]

labels_train_loaded = [labels_list[ind] for ind in indices_train]
labels_test_loaded = [labels_list[ind] for ind in indices_test]

print('len of videos_train: ', len(videos_train))
print('len of videos_test: ', len(videos_test))
print('len of labels_train_loaded: ', len(labels_train_loaded))
print('len of labels_test_loaded: ', len(labels_test_loaded))


len of videos_train:  80
len of videos_test:  24
len of labels_train_loaded:  80
len of labels_test_loaded:  24


In [4]:
def get_labels_from_video(no_frames, safe_duration_list):
    
    '''
    Get labels for a particular video 

    Parameters:
    no_frames(int) : no of frames in the given video
    safe_duration_list(list) : a list of the type [safe_start1, safe_end1, safe_start2, safe_end2,......]

    Returns:
    list : list with len = no of frames and the value at each index represents safe/unsafe at that frame_no (frame_no starting at 0)
    int : -1 if there is no safe duration in video, 1 otherwise
    '''

    labels = [0]*no_frames
    no_safe_durations = int(len(safe_duration_list)/2)
    if(no_safe_durations == 0):
        return labels,-1 # there is no safe duration in the given video so all labels marked 0
    else:

        for i in range(no_safe_durations):
            safe_start = max(safe_duration_list[i*2] - 1, 0)
            safe_end = min(safe_duration_list[i*2 +1] - 1, no_frames-1)
            labels[safe_start:safe_end+1] = [1]*(safe_end-safe_start+1) # marking the value b/w safe_start and safe_end with 1

    if len(labels) > no_frames: #len of labels cannot be greater than no_frames in video
        raise Exception('Check the labels assigned in CSV file!')
    return labels,1

In [5]:
labels_train = []
labels_test = []

for i in range(len(videos_train[:5])):

    print(videos_train[i])
    cap = cv2.VideoCapture(videos_train[i])
    no_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    l, f = get_labels_from_video(no_frames, labels_train_loaded[i])
    labels_train.extend(l)

for i in range(len(videos_test)):

    print(videos_test[i])
    cap = cv2.VideoCapture(videos_test[i])
    no_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    l, f = get_labels_from_video(no_frames, labels_test_loaded[i])
    labels_test.extend(l)

labels_train = np.array(labels_train)
labels_test = np.array(labels_test)

print("shape of labels_train", labels_train.shape)
print("shape of labels_test", labels_test.shape)

C:/RoadCrossingAssistant/Data/Videos\video1.MOV
C:/RoadCrossingAssistant/Data/Videos\video2.MOV
C:/RoadCrossingAssistant/Data/Videos\video3.MOV
C:/RoadCrossingAssistant/Data/Videos\video4.MOV
C:/RoadCrossingAssistant/Data/Videos\video6.MOV
C:/RoadCrossingAssistant/Data/Videos\video28.MOV
C:/RoadCrossingAssistant/Data/Videos\video66.MOV
C:/RoadCrossingAssistant/Data/Videos\video47.MOV
C:/RoadCrossingAssistant/Data/Videos\video36.MOV
C:/RoadCrossingAssistant/Data/Videos\video19.MOV
C:/RoadCrossingAssistant/Data/Videos\video23.MOV
C:/RoadCrossingAssistant/Data/Videos\video42.MOV
C:/RoadCrossingAssistant/Data/Videos\video18.MOV
C:/RoadCrossingAssistant/Data/Videos\video69.MOV
C:/RoadCrossingAssistant/Data/Videos\video35.MOV
C:/RoadCrossingAssistant/Data/Videos\video102.MOV
C:/RoadCrossingAssistant/Data/Videos\video17.MOV
C:/RoadCrossingAssistant/Data/Videos\video40.MOV
C:/RoadCrossingAssistant/Data/Videos\video85.MOV
C:/RoadCrossingAssistant/Data/Videos\video21.MOV
C:/RoadCrossingAssistant

In [6]:
#get required frame indices

def get_required_indices(labels):

    ind0 = np.where(labels == 0)[0]
    ind1 = np.where(labels == 1)[0]

    print(ind0.shape, ind1.shape)

    if (len(ind0)/len(ind1)) > 1.2:
        print('reducing the number of unsafe frames\n\n')
        len_ind0 = int(len(ind1)*1.2)
        ind0 = ind0[:len_ind0]

        indices_required = np.concatenate((ind0, ind1))
    
    return indices_required

indices_required_train = get_required_indices(labels_train)
indices_required_test = get_required_indices(labels_test)

labels_train_reduced = labels_train[indices_required_train]
labels_test_reduced = labels_test[indices_required_test]

print("shape on labels_train_reduced", labels_train_reduced.shape)
print("shape of labels_test_reduced", labels_test_reduced.shape)



(764,) (136,)
reducing the number of unsafe frames


(3937,) (1913,)
reducing the number of unsafe frames


shape on labels_train_reduced (299,)
shape of labels_test_reduced (4208,)


In [7]:

def get_required_frames(videos, indices_required):

    frames = []
    ind = -1

    for i in range(len(videos)):

        print(videos[i])
        cap = cv2.VideoCapture(videos[i])
        no_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        #print(no_frames)
        
        while cap.isOpened() :
            success, frame = cap.read()
            if not success:
                break
            ind = ind + 1
            if ind in indices_required:
                #frames_train.append(frame)
                #print(frame.shape)
                frame_resized = cv2.resize(frame, (360, 640), interpolation = cv2.INTER_AREA)
                #print(frame_resized.shape)
                frames.append(frame_resized)
    return frames

frames_train = get_required_frames(videos_train, indices_required_train)
frames_test = get_required_frames(videos_test, indices_required_test)

print("shape of frames_train: ",frames_train.shape)
print("shape of frames_test: ",frames_test.shape)

        


C:/RoadCrossingAssistant/Data/Videos\video1.MOV
C:/RoadCrossingAssistant/Data/Videos\video2.MOV
C:/RoadCrossingAssistant/Data/Videos\video3.MOV
C:/RoadCrossingAssistant/Data/Videos\video4.MOV
C:/RoadCrossingAssistant/Data/Videos\video6.MOV
(299, 640, 360, 3)


In [39]:
1920/3,1080/3

(640.0, 360.0)